In [1]:
import pandas as pd
from mongo_client import fetch_listings  # your existing module

In [2]:
# Loa# Load the data into df
df = fetch_listings(limit=6000)
print("Loaded:", df.shape)

Loaded: (5555, 42)


In [3]:
# 1. Remove duplicates
df = df.drop_duplicates(subset=["_id"], keep="first")
print("After duplicates:", df.shape)

After duplicates: (5555, 42)


In [4]:
df.dtypes


_id                       object
listing_url               object
name                      object
summary                   object
space                     object
description               object
neighborhood_overview     object
notes                     object
transit                   object
access                    object
interaction               object
house_rules               object
property_type             object
room_type                 object
bed_type                  object
minimum_nights            object
maximum_nights            object
cancellation_policy       object
last_scraped              object
calendar_last_scraped     object
first_review              object
last_review               object
accommodates               int64
bedrooms                 float64
beds                     float64
number_of_reviews          int64
bathrooms                float64
amenities                 object
price                    float64
security_deposit         float64
cleaning_f

In [5]:
print(df.info())  # Shows number of non-null entries, column types, and memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5555 entries, 0 to 5554
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _id                    5555 non-null   object 
 1   listing_url            5555 non-null   object 
 2   name                   5555 non-null   object 
 3   summary                5555 non-null   object 
 4   space                  5555 non-null   object 
 5   description            5555 non-null   object 
 6   neighborhood_overview  5555 non-null   object 
 7   notes                  5555 non-null   object 
 8   transit                5555 non-null   object 
 9   access                 5555 non-null   object 
 10  interaction            5555 non-null   object 
 11  house_rules            5555 non-null   object 
 12  property_type          5555 non-null   object 
 13  room_type              5555 non-null   object 
 14  bed_type               5555 non-null   object 
 15  mini

In [6]:
#Convert date columns to datetime
date_cols = ['last_scraped', 'calendar_last_scraped', 'first_review', 'last_review']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')  # errors='coerce' sets invalid dates to NaT

In [7]:
# Convert numeric columns from object to integer
df['minimum_nights'] = pd.to_numeric(df['minimum_nights'], errors='coerce')
df['maximum_nights'] = pd.to_numeric(df['maximum_nights'], errors='coerce')

In [8]:
# Now check the data types
print(df.dtypes)

_id                              object
listing_url                      object
name                             object
summary                          object
space                            object
description                      object
neighborhood_overview            object
notes                            object
transit                          object
access                           object
interaction                      object
house_rules                      object
property_type                    object
room_type                        object
bed_type                         object
minimum_nights                    int64
maximum_nights                    int64
cancellation_policy              object
last_scraped             datetime64[ns]
calendar_last_scraped    datetime64[ns]
first_review             datetime64[ns]
last_review              datetime64[ns]
accommodates                      int64
bedrooms                        float64
beds                            float64


Handling Missing Values

In [9]:
# 1. For bedrooms, beds, bathrooms: fill with median (accurate, preserves distribution)
df['bedrooms'].fillna(df['bedrooms'].median(), inplace=True)
df['beds'].fillna(df['beds'].median(), inplace=True)
df['bathrooms'].fillna(df['bathrooms'].median(), inplace=True)


In [10]:
# 2. For security_deposit, cleaning_fee, weekly_price, monthly_price:
# These are often missing if not charged, so fill with 0  (assume missing means not applicable).
def monetary_columns():
    return ['security_deposit', 'cleaning_fee', 'weekly_price', 'monthly_price']
df[monetary_columns()] = df[monetary_columns()].fillna(0)



In [11]:
# 3. reviews_per_month: Treat NaN as 0 = no reviews received
df['reviews_per_month'].fillna(0, inplace=True)



In [12]:
# 4. For first_review and last_review:
# These are useful for time-based analysis (seasonal availability, demand trends)
# So keep them and fill with a neutral placeholder date to retain rows for completeness
df['first_review'].fillna(pd.Timestamp('2000-01-01'), inplace=True)
df['last_review'].fillna(pd.Timestamp('2000-01-01'), inplace=True)



In [13]:
# Check if missing values are handled completely
print("\nMissing values after cleaning:")
print(df.isnull().sum()[df.isnull().sum() > 0])


Missing values after cleaning:
Series([], dtype: int64)


In [14]:
print(df.columns.tolist())


['_id', 'listing_url', 'name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'property_type', 'room_type', 'bed_type', 'minimum_nights', 'maximum_nights', 'cancellation_policy', 'last_scraped', 'calendar_last_scraped', 'first_review', 'last_review', 'accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'bathrooms', 'amenities', 'price', 'security_deposit', 'cleaning_fee', 'extra_people', 'guests_included', 'images', 'host', 'address', 'availability', 'review_scores', 'reviews', 'weekly_price', 'monthly_price', 'reviews_per_month']


Feature Engineering - Flattening nested fields for EDA

In [15]:
# Extract latitude and longitude from 'address.location.coordinates'
df['longitude'] = df['address'].apply(lambda x: x.get('location', {}).get('coordinates', [None, None])[0])
df['latitude'] = df['address'].apply(lambda x: x.get('location', {}).get('coordinates', [None, None])[1])

In [16]:
# Extract city and country 

df['city'] = df['address'].apply(
    lambda x: x.get('market') if isinstance(x, dict) and 'market' in x 
    else x.get('city', None) if isinstance(x, dict) 
    else None
)
df['country'] = df['address'].apply(
    lambda x: x.get('country', None) if isinstance(x, dict) 
    else None
)


In [17]:
print(df['city'].unique())
print(df['city'].isnull().sum())


['New York' 'Istanbul' 'Hong Kong' 'Sydney' 'Porto' 'Rio De Janeiro'
 'Montreal' 'Barcelona' 'Oahu' 'The Big Island' 'Maui' 'Kauai' ''
 'Other (Domestic)' 'Other (International)']
0


In [18]:
# Drop rows where city is missing or empty
df = df[df['city'].replace('', pd.NA).notnull()]
print("✅ Dropped rows with missing city. New shape:", df.shape)



✅ Dropped rows with missing city. New shape: (5549, 46)


In [19]:
# Extract host name and host id
df['host_name'] = df['host'].apply(lambda x: x.get('host_name', None))
df['host_id'] = df['host'].apply(lambda x: x.get('host_id', None))



In [20]:
# Extract review scores if available
if 'review_scores' in df.columns:
    df['review_rating'] = df['review_scores'].apply(lambda x: x.get('review_scores_rating', None))
    df['review_accuracy'] = df['review_scores'].apply(lambda x: x.get('review_scores_accuracy', None))
    df['review_cleanliness'] = df['review_scores'].apply(lambda x: x.get('review_scores_cleanliness', None))




In [75]:
# Drop complex columns that won’t be used directly in EDA
#columns_to_drop = ['images', 'host', 'review_scores', 'reviews']
#df.drop(columns=columns_to_drop, inplace=True, errors='ignore')


In [21]:

# Final check on new shape and columns
print("\nTransformed DataFrame Shape:", df.shape)
print("\nUpdated Columns:", df.columns.tolist())


Transformed DataFrame Shape: (5549, 51)

Updated Columns: ['_id', 'listing_url', 'name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'property_type', 'room_type', 'bed_type', 'minimum_nights', 'maximum_nights', 'cancellation_policy', 'last_scraped', 'calendar_last_scraped', 'first_review', 'last_review', 'accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'bathrooms', 'amenities', 'price', 'security_deposit', 'cleaning_fee', 'extra_people', 'guests_included', 'images', 'host', 'address', 'availability', 'review_scores', 'reviews', 'weekly_price', 'monthly_price', 'reviews_per_month', 'longitude', 'latitude', 'city', 'country', 'host_name', 'host_id', 'review_rating', 'review_accuracy', 'review_cleanliness']


In [22]:
df['longitude'] = df['address'].apply(lambda x: x.get('location', {}).get('coordinates', [None, None])[0])
df['latitude'] = df['address'].apply(lambda x: x.get('location', {}).get('coordinates', [None, None])[1])

In [23]:
# Ensure date columns are datetime type
df['first_review'] = pd.to_datetime(df['first_review'], errors='coerce')
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

In [24]:
print(df.dtypes)

_id                              object
listing_url                      object
name                             object
summary                          object
space                            object
description                      object
neighborhood_overview            object
notes                            object
transit                          object
access                           object
interaction                      object
house_rules                      object
property_type                    object
room_type                        object
bed_type                         object
minimum_nights                    int64
maximum_nights                    int64
cancellation_policy              object
last_scraped             datetime64[ns]
calendar_last_scraped    datetime64[ns]
first_review             datetime64[ns]
last_review              datetime64[ns]
accommodates                      int64
bedrooms                        float64
beds                            float64


In [25]:
df['review_rating'].fillna(0, inplace=True)
df['review_accuracy'].fillna(0, inplace=True)
df['review_cleanliness'].fillna(0, inplace=True)


In [26]:
# Check if missing values are handled completely
print("\nMissing values after cleaning:")
print(df.isnull().sum()[df.isnull().sum() > 0])


Missing values after cleaning:
Series([], dtype: int64)


In [97]:
'''import os
import pandas as pd

# File path
file_path = "cleaned_airbnb_data.csv"

# Delete the old file if it exists
if os.path.exists(file_path):
    os.remove(file_path)
    print("✅ Old CSV file deleted.")
else:
    print("⚠️ No old file found.")'''


'import os\nimport pandas as pd\n\n# File path\nfile_path = "cleaned_airbnb_data.csv"\n\n# Delete the old file if it exists\nif os.path.exists(file_path):\n    os.remove(file_path)\n    print("✅ Old CSV file deleted.")\nelse:\n    print("⚠️ No old file found.")'

In [ ]:
# Save the cleaned DataFrame again
'''df.to_csv("cleaned_airbnb_data.csv", index=False)
print("✅ Final cleaned CSV file saved.")'''


✅ Final cleaned CSV file saved.


In [28]:
print(df.columns.tolist())


['_id', 'listing_url', 'name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'property_type', 'room_type', 'bed_type', 'minimum_nights', 'maximum_nights', 'cancellation_policy', 'last_scraped', 'calendar_last_scraped', 'first_review', 'last_review', 'accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'bathrooms', 'amenities', 'price', 'security_deposit', 'cleaning_fee', 'extra_people', 'guests_included', 'images', 'host', 'address', 'availability', 'review_scores', 'reviews', 'weekly_price', 'monthly_price', 'reviews_per_month', 'longitude', 'latitude', 'city', 'country', 'host_name', 'host_id', 'review_rating', 'review_accuracy', 'review_cleanliness']
